In [1]:
%%capture
!pip install transformers
import pandas as pd
from transformers import pipeline

# Initialize the fill-mask pipeline
unmasker = pipeline('fill-mask', model='microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')

In [2]:
disease_df = pd.read_excel('/content/icd_transformers.xlsx')

In [ ]:
#unmasker = pipeline('fill-mask',
#                    model='emilyalsentzer/Bio_ClinicalBERT',
#                    tokenizer='emilyalsentzer/Bio_ClinicalBERT')

In [3]:
import pandas as pd

# Set the number of rows to read
num_rows = int(pd.read_excel('/content/icd_transformers.xlsx').shape[0] * 1)

# Read the first 1% of rows
disease_df = pd.read_excel('/content/icd_transformers.xlsx', nrows=num_rows)

# Print the first few rows to verify the data was loaded correctly
print(disease_df.head())


      ICD10                                            disease
0  e09.3219  Drug or chemical induced diabetes mellitus wit...
1     e31.8                    Other polyglandular dysfunction
2     e67.3                                 Hypervitaminosis D
3   e71.518           Other disorders of peroxisome biogenesis
4     e77.0  Defects in post-translational modification of ...


In [4]:
disease_df

,ICD10,disease
0,e09.3219,Drug or chemical induced diabetes mellitus wit...
1,e31.8,Other polyglandular dysfunction
2,e67.3,Hypervitaminosis D
3,e71.518,Other disorders of peroxisome biogenesis
4,e77.0,Defects in post-translational modification of ...
...,...,...
424,z83.2,Family history of diseases of the blood and bl...
425,z85.29,Personal history of malignant neoplasm of othe...
426,z85.810,Personal history of malignant neoplasm of tongue
427,z86.74,Personal history of sudden cardiac arrest


In [ ]:
#obsolete
import psutil
from tqdm import tqdm

# Create a list of all possible disease names
disease_list = disease_df['disease']

# Initialize an empty dictionary to store the filled-in sentences
sentences_dict = {}

# Calculate the total number of iterations needed for the progress bar
total_iterations = len(disease_list) * 20

# Initialize the progress bar
pbar = tqdm(total=total_iterations)

# Generate filled-in sentences for all diseases
for disease in disease_list:
    sentence = f"[MASK] is a medication used in treatment of {disease}"
    filled_sentences = unmasker(sentence, top_k=20)
    if disease not in sentences_dict:
        sentences_dict[disease] = {f"Drug {j+1}": "" for j in range(20)}
    for i in range(len(filled_sentences)):
        if i >= 20:
            break
        if sentences_dict[disease][f"Drug {i+1}"] == "":
            drug_name = filled_sentences[i]['token_str']
            probability = filled_sentences[i]['score']
            sentences_dict[disease][f"Drug {i+1}"] = f"{drug_name} ({probability})"
        # Update the progress bar
        pbar.update(1)

        # Check memory usage and print a warning if RAM usage exceeds 90%
        ram_usage = psutil.virtual_memory().percent
        if ram_usage > 90:
            print("Warning: RAM usage exceeds 90%")

# Create a new dataframe from the dictionary of filled-in sentences
result_df = pd.DataFrame.from_dict(sentences_dict, orient='index')

# Add the 'ICD10' column from the input dataframe to the output dataframe
result_df['ICD10'] = disease_df['ICD10']

# Print the resulting dataframe
print(result_df)

# Close the progress bar
pbar.close()

In [5]:
import psutil
from tqdm import tqdm

# Create a list of all possible disease names
disease_list = disease_df['disease']

# Initialize an empty dictionary to store the filled-in sentences
sentences_dict = {}

# Calculate the total number of iterations needed for the progress bar
total_iterations = len(disease_list) * 20

# Initialize the progress bar
pbar = tqdm(total=total_iterations)

# Generate filled-in sentences for all diseases
for disease in disease_list:
    sentence = f"[MASK] is an antidote used it treating {disease}"
    filled_sentences = unmasker(sentence, top_k=20)
    if disease not in sentences_dict:
        sentences_dict[disease] = {f"Drug {j+1}": "" for j in range(20)}
    for i in range(len(filled_sentences)):
        if i >= 20:
            break
        if sentences_dict[disease][f"Drug {i+1}"] == "":
            drug_name = filled_sentences[i]['token_str']
            probability = filled_sentences[i]['score']
            sentences_dict[disease][f"Drug {i+1}"] = f"{drug_name} ({probability})"
        # Update the progress bar
        pbar.update(1)

        # Check memory usage and print a warning if RAM usage exceeds 90%
        ram_usage = psutil.virtual_memory().percent
        if ram_usage > 90:
            print("Warning: RAM usage exceeds 90%")

# Create a new dataframe from the dictionary of filled-in sentences
result_df = pd.DataFrame(sentences_dict).transpose()

# Rename the index column to 'disease'
result_df.index.name = 'disease'

# Reset the index of the result dataframe
result_df.reset_index(inplace=True)

# Join the 'ICD10' and 'disease' columns from the input dataframe to the output dataframe
result_df = pd.merge(result_df, disease_df[['ICD10', 'disease']], on='disease') ############## revoke this if you need.

# Print the resulting dataframe
print(result_df)

# Close the progress bar
pbar.close()

100%|██████████| 8580/8580 [01:18<00:00, 109.89it/s]

                                               disease  \
0    Drug or chemical induced diabetes mellitus wit...   
1                      Other polyglandular dysfunction   
2                                   Hypervitaminosis D   
3             Other disorders of peroxisome biogenesis   
4    Defects in post-translational modification of ...   
..                                                 ...   
424  Family history of diseases of the blood and bl...   
425  Personal history of malignant neoplasm of othe...   
426   Personal history of malignant neoplasm of tongue   
427          Personal history of sudden cardiac arrest   
428               Cataract extraction status, left eye   

                       Drug 1                             Drug 2  \
0     it (0.6107997894287109)         this (0.05407034978270531)   
1     it (0.5859982371330261)     atropine (0.05934412404894829)   
2    it (0.48169374465942383)        this (0.037143100053071976)   
3     it (0.4138701856136322)  

In [6]:
result_df

,disease,Drug 1,Drug 2,Drug 3,Drug 4,Drug 5,Drug 6,Drug 7,Drug 8,Drug 9,...,Drug 12,Drug 13,Drug 14,Drug 15,Drug 16,Drug 17,Drug 18,Drug 19,Drug 20,ICD10
0,Drug or chemical induced diabetes mellitus wit...,it (0.6107997894287109),this (0.05407034978270531),metformin (0.05300213769078255),insulin (0.020464176312088966),ada (0.007964986376464367),there (0.00649842107668519),he (0.0039465539157390594),which (0.0034806535113602877),atropine (0.002965005347505212),...,they (0.0023579124826937914),drug (0.0021329366136342287),atorvastatin (0.002069271868094802),dexamethasone (0.0020523990970104933),statin (0.002004497917369008),paracetamol (0.0019264755537733436),warfarin (0.0019109403947368264),gabapentin (0.0017269144300371408),na (0.0017081003170460463),e09.3219
1,Other polyglandular dysfunction,it (0.5859982371330261),atropine (0.05934412404894829),this (0.025711387395858765),paracetamol (0.022707708179950714),propranolol (0.015906812623143196),naloxone (0.010641316883265972),diazepam (0.010523592121899128),there (0.010167435742914677),caffeine (0.008276740089058876),...,colchicine (0.004373334813863039),haloperidol (0.004315349273383617),clonidine (0.003987826872617006),ketamine (0.003913060296326876),he (0.003390090772882104),which (0.003153180703520775),methadone (0.0029723038896918297),verapamil (0.0028926744125783443),acetaminophen (0.0026790269184857607),e31.8
2,Hypervitaminosis D,it (0.48169374465942383),this (0.037143100053071976),vitamin (0.03446371108293533),thiamine (0.020275866612792015),warfarin (0.01576845347881317),calcium (0.014725355431437492),edta (0.013943055644631386),biotin (0.013515749014914036),b12 (0.011690559796988964),...,iron (0.006927788723260164),potassium (0.005977984983474016),magnesium (0.005490653216838837),carnitine (0.005459023639559746),there (0.005397559609264135),aluminium (0.005254785064607859),propranolol (0.004895887803286314),they (0.004318636376410723),sodium (0.0042546712793409824),e67.3
3,Other disorders of peroxisome biogenesis,it (0.4138701856136322),metformin (0.029851442202925682),this (0.02831238880753517),carnitine (0.020689424127340317),pioglitazone (0.012548877857625484),chloroquine (0.011676622554659843),caffeine (0.011567603796720505),there (0.009221196174621582),colchicine (0.00905307661741972),...,atorvastatin (0.007332684006541967),rosiglitazone (0.006800839677453041),isoniazid (0.006512582302093506),resveratrol (0.006000140681862831),doxycycline (0.005554374773055315),paracetamol (0.0054355221800506115),sorafenib (0.004890223499387503),rapamycin (0.004743651021271944),propranolol (0.004528755322098732),e71.518
4,Defects in post-translational modification of ...,it (0.3909582793712616),chloroquine (0.16357140243053436),ert (0.07267767190933228),this (0.019987979903817177),ribavirin (0.015438033267855644),bortezomib (0.012011256068944931),colchicine (0.00893089547753334),carnitine (0.008026067167520523),azt (0.007042760495096445),...,lithium (0.006037733051925898),gaa (0.005842558108270168),leupeptin (0.005594165530055761),rituximab (0.005507183261215687),there (0.00509747164323926),carbamazepine (0.004381774924695492),methotrexate (0.004258686676621437),they (0.0040196808986365795),lamivudine (0.00347092654556036),e77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Family history of diseases of the blood and bl...,it (0.558965802192688),this (0.025617750361561775),heparin (0.018995095044374466),edta (0.01684844121336937),warfarin (0.015595999546349049),chloroquine (0.011749731376767159),paracetamol (0.011446243152022362),aspirin (0.011245038360357285),penicillin (0.00839292723685503),...,propranolol (0.006583984475582838),colchicine (0.006200903095304966),immunoglobulin (0.005932249594479799),cyclophosphamide (0.005460152868181467),arsenic (0.005352123640477657),interferon (0.0051727294921875),rifampicin (0.004861151333898306),he (0.0044908891431987286),atropine (0.004080372862517834),z83.2
425,Personal history of malignant ne

In [7]:
result_df.to_excel('/content/drugs_icd_transformers_2.xlsx')

In [ ]:
#obsolete
import pandas as pd

# read in the dataframe
df = pd.read_excel('/content/drugs_icd_transformers_2.xlsx')

# split the drug columns by space character and extract only the drug names
drug_cols = ['Drug {}'.format(i) for i in range(1, 21)]
for col in drug_cols:
    df[col] = df[col].str.split(' ', 1, expand=True)[0]

# write the updated dataframe to a new excel file
df.to_excel('/content/drugs_icd_transformers_2_no_prob_score.xlsx', index=False)

In [ ]:
#obsolete
import pandas as pd

# read in the dataframe
df = pd.read_excel('/content/drugs_icd_transformers_2.xlsx')

# split the drug columns by space character into drug name and probability columns
drug_cols = ['Drug {}'.format(i) for i in range(1, 21)]
for col in drug_cols:
    df[[col + ' Name', col + ' Probability']] = df[col].str.split(' ', 1, expand=True)

# write the updated dataframe to a new excel file
df.to_excel('/content/drugs_icd_transformers_2_split_prob_score.xlsx', index=False)

In [8]:
import pandas as pd

# read in the dataframe
df = pd.read_excel('/content/drugs_icd_transformers_2.xlsx')

# split the drug columns by space character into drug name and probability columns
drug_cols = ['Drug {}'.format(i) for i in range(1, 21)]
for col in drug_cols:
    df[col + ' Probability'] = df[col].str.extract('\((.*?)\)', expand=True)
    df[col] = df[col].str.replace(r'\(.*?\)', '').str.strip()

# write the updated dataframe to a new excel file
df.to_excel('/content/drugs_icd_transformers_2_split_prob_score.xlsx', index=False)

<ipython-input-8-baf34a16ffbb>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(.*?\)', '').str.strip()
<ipython-input-8-baf34a16ffbb>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(.*?\)', '').str.strip()
<ipython-input-8-baf34a16ffbb>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(.*?\)', '').str.strip()
<ipython-input-8-baf34a16ffbb>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(.*?\)', '').str.strip()
<ipython-input-8-baf34a16ffbb>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(.*?\)', '').str.strip()
<ipython-input-8-baf34a16ffbb>:10: FutureWarning: The d

In [9]:
import pandas as pd
import numpy as np
import re
# read in the original dataframe
df = pd.read_excel('/content/drugs_icd_transformers_2_split_prob_score.xlsx')

# read in the values to drop dataframe
to_drop_df = pd.read_excel('/content/values_to_drop.xlsx')



In [10]:
# Convert the values_to_drop dataframe to a list of strings
values_to_drop = list(map(str, to_drop_df.iloc[:, 0]))

# Escape any special characters in the values to drop
values_to_drop = [re.escape(value) for value in values_to_drop]

# Create a regular expression pattern to match any of the values to drop when they appear as entire words
pattern = r'\b(?:{})\b'.format('|'.join(values_to_drop))
df = df.replace(pattern, '', regex=True)

In [11]:
df

,Unnamed: 0,disease,Drug 1,Drug 2,Drug 3,Drug 4,Drug 5,Drug 6,Drug 7,Drug 8,...,Drug 11 Probability,Drug 12 Probability,Drug 13 Probability,Drug 14 Probability,Drug 15 Probability,Drug 16 Probability,Drug 17 Probability,Drug 18 Probability,Drug 19 Probability,Drug 20 Probability
0,0,Drug chemical induced diabetes mellitus mild ...,,,metformin,insulin,ada,,,,...,0.002378,0.002358,0.002133,0.002069,0.002052,0.002004,0.001926,0.001911,0.001727,0.001708
1,1,Other polyglandular dysfunction,,atropine,,paracetamol,propranolol,naloxone,diazepam,,...,0.006121,0.004373,0.004315,0.003988,0.003913,0.003390,0.003153,0.002972,0.002893,0.002679
2,2,Hypervitaminosis D,,,,thiamine,warfarin,calcium,edta,biotin,...,0.008713,0.006928,0.005978,0.005491,0.005459,0.005398,0.005255,0.004896,0.004319,0.004255
3,3,Other disorders peroxisome biogenesis,,metformin,,carnitine,pioglitazone,chloroquine,caffeine,,...,0.007625,0.007333,0.006801,0.006513,0.006000,0.005554,0.005436,0.004890,0.004744,0.004529
4,4,Defects post-translational modification lysos...,,chloroquine,,,ribavirin,bortezomib,colchicine,carnitine,...,0.006840,0.006038,0.005843,0.005594,0.005507,0.005097,0.004382,0.004259,0.004020,0.003471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,Family diseases -forming organs certain ...,,,heparin,edta,warfarin,chloroquine,paracetamol,aspirin,...,0.006726,0.006584,0.006201,0.005932,0.005460,0.005352,0.005173,0.004861,0.004491,0.004080
425,425,Personal malignant neoplasm intrathoracic ...,,,bleomycin,cisplatin,propranolol,,atropine,warfarin,...,0.007878,0.007201,0.006426,0.006412,0.006374,0.006205,0.005968,0.005962,0.005389,0.005290
426,426,Personal malignant neoplasm tongue,,,propranolol,atropine,paracetamol,warfarin,edta,cisplatin,...,0.006370,0.006333,0.005501,0.005496,0.005495,0.005197,0.004653,0.004455,0.004136,0.003968
427,427,Personal sudden cardiac arrest,,atropine,epinephrine,,propranolol,naloxone,ecg,digoxin,...,0.008456,0.007627,0.006345,0.006278,0.006077,0.005984,0.005935,0.005144,0.004578,0.004420


In [12]:
df.to_excel('/content/updated_drugs_for_cm_version.xlsx', index=False)

In [ ]:
result_df.to_excel('drugs_icd_transformers_2.xlsx')

In [ ]:
unmasker_dict = {
    'microsoft': pipeline('fill-mask', model='microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'),
    'dmis-lab': pipeline('fill-mask', model='dmis-lab/biobert-base-cased-v1.1'),
    'emilyalsentzer': pipeline('fill-mask', model='emilyalsentzer/Bio_ClinicalBERT'),
    'scibert': pipeline('fill-mask', model='gsarti/scibert-nli')
}



In [ ]:
# List of pre-trained models to use
models = ['microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext',
          'dmis-lab/biobert-base-cased-v1.1',
          'allenai/scibert_scivocab_cased',
          'microsoft/BiomedNLP-PubMedBERT-base-cased-abstract',
          'emilyalsentzer/Bio_ClinicalBERT']

In [ ]:
import psutil
from tqdm import tqdm
from transformers import pipeline


# Create a list of all possible disease names
disease_list = disease_df['disease']

# Initialize an empty dictionary to store the filled-in sentences
sentences_dict = {}

# Calculate the total number of iterations needed for the progress bar
total_iterations = len(disease_list) * len(models) * 10

# Initialize the progress bar
pbar = tqdm(total=total_iterations)

# Generate filled-in sentences for all diseases
for disease in disease_list:
    for model in models:
        sentence = f"[MASK] is medication used in treatment of {disease}"
        filled_sentences = unmasker_dict[model](sentence, top_k=10)
        if disease not in sentences_dict:
            sentences_dict[disease] = {f"Drug {j+1} ({model})": "" for j in range(10)}
        for i in range(len(filled_sentences)):
            if i >= 10:
                break
            if sentences_dict[disease][f"Drug {i+1} ({model})"] == "":
                drug_name = filled_sentences[i]['token_str']
                probability = filled_sentences[i]['score']
                sentences_dict[disease][f"Drug {i+1} ({model})"] = f"{drug_name} ({probability})"
            # Update the progress bar
            pbar.update(1)

            # Check memory usage and print a warning if RAM usage exceeds 90%
            ram_usage = psutil.virtual_memory().percent
            if ram_usage > 90:
                print("Warning: RAM usage exceeds 90%")

# Create a new dataframe from the dictionary of filled-in sentences
result_df = pd.DataFrame(sentences_dict).transpose()

# Print the resulting dataframe
print(result_df)

# Close the progress bar
pbar.close()


In [ ]:
!pip install drawdata

In [3]:
import pandas as pd

In [4]:
from drawdata import draw_scatter

draw_scatter()